In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import ccxt
from binance.client import Client
import pandas as pd
import numpy as np
from pathlib import Path
import time

### Global Variable

In [2]:
binance = ccxt.binance()
APIKEY = 'Ms9e5Ls1AfaX4fMALsg2gPEOaEhB2qlJhWTzViK7eSxYuylYuBmHukteRn6GDaWj'
## SECRETKEY = ''
client = Client(APIKEY,SECRETKEY)

In [3]:
target = 'BTCUSDT'
start_date = '15 Aug, 2017'
end_date = '31 Aug, 2021'
feature_name = f'{target} {start_date} {end_date}.csv'

### Path Settings

In [4]:
feature_name

'BTCUSDT 15 Aug, 2017 31 Aug, 2021.csv'

In [5]:
feature_dir = Path('../CUD/feature')
feature_file = feature_dir / feature_name

### Get Data

In [6]:
klines = client.get_historical_klines(target, Client.KLINE_INTERVAL_4HOUR, start_date, end_date)

In [7]:
col_name = ['Open time','Open','High','Low','Close','Volume',
           'Close time','Quote asset volume','Num trades',
           'bav','qav','ignored']
df = pd.DataFrame(klines, columns = col_name)

### Feature Engineering

In [8]:
# drop unnecessary data
df.drop(['Close time','bav','qav','ignored'],axis=1,inplace=True)

In [9]:
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.6f}'.format

df = df.astype('float')

In [10]:
# transform readable data
def trans_time(a):
    temp = a // 1000
    return time.ctime(temp)

df['Open time'] = df['Open time'].apply(trans_time)

### Add Feature

In [11]:
def VWAP(df):
    vol = df.Volume.values
    p = (df.Low + df.Close + df.High).div(3).values
    
    return (p*vol).cumsum() / vol.cumsum()

df['VWAP'] = VWAP(df)

In [12]:
def RSI(df,period):
    U = np.where(df.Close.diff(1) > 0, df.Close.diff(1), 0)
    D = np.where(df.Close.diff(1) < 0, df.Close.diff(1)*(-1), 0)
    AU = pd.DataFrame(U).rolling(window=period).mean()
    AD = pd.DataFrame(D).rolling(window=period).mean()
    RSI = AU / (AD+AU) * 100
    
    return RSI

df['RSI'] = RSI(df,14)

### Send Data

In [13]:
feature_name+'.csv'

'BTCUSDT 15 Aug, 2017 31 Aug, 2021.csv.csv'

In [14]:
df.to_csv(feature_file,index = False)